In [1]:
import pandas as pd
import numpy as np

from datetime import timedelta,datetime
import datetime
from calendar import monthrange
import calendar

from dateutil.relativedelta import relativedelta
import math

import csv

In [2]:
new_data_raw=pd.read_excel("D:\\vikas\\office\\castlight\\standalone\\upload_v1.xlsx")

#existing_allocation_raw=pd.read_csv("D:\\office\\castlight_v3\\generated_summary\\from_retrospective\\final_retro_allocation.csv")
#existing_monthly_allocation_raw=pd.read_csv("D:\\office\\castlight_v3\\generated_summary\\from_retrospective\\final_retro_monthly_allocation.csv")

#existing_allocation_raw=pd.read_csv("D:\\office\\castlight_v3\\upload_line\\prospective_summary_09_06.csv")
existing_allocation_raw=pd.read_csv("D:\\vikas\\office\\castlight_v3\\upload_line\\for_pros_summary_09_07.csv")
#existing_monthly_allocation_raw=pd.read_csv("D:\\office\\castlight_v3\\upload_line\\prospective_monthly_summary_09_06.csv")
existing_monthly_allocation_raw=pd.read_csv("D:\\vikas\\office\\castlight_v3\\upload_line\\for_pros_monthly_0915.csv")

In [3]:
existing_allocation_raw.shape
existing_monthly_allocation_raw.shape

(202, 6)

In [4]:
new_data_raw = new_data_raw[new_data_raw['Allocation Type']=='Prospective']

In [5]:
upload_date=datetime.datetime(2015, 8, 1).date()

In [6]:
#Define funtion to get month difference between 2 dates. To be used to calculate allocation term
def get_allocation_term(term,contract_start_date,contract_end_date,allocation_start_date,allocation_end_date):
    alloc_term=round(term*(abs((allocation_end_date - allocation_start_date).days))/(abs((contract_end_date - contract_start_date).days)),2)
    number_dec = round(float(str(alloc_term-int(alloc_term))[1:]),2)
    if number_dec == 0.99:
        return alloc_term + 0.01
    else:
        return alloc_term

#Define funtion to calculate daily ratable revenue
def get_ratable_revenue(ext_ssp_price,start_date,end_date):
    return ext_ssp_price/(abs((end_date - start_date).days)+1)

#Define funtion to fetch number of days in each month between 2 dates. To be used to calculate monthly revenue table from allocation summary table
def get_allocation_durations(start_date,end_date):
    current = start_date
    result = [current]

    current = current.replace(day=1)
    while current <= end_date:
        current += timedelta(days=32)
        current = current.replace(day=1)
        result.append(datetime.datetime(current.year, current.month, 1).date())
    
    durations= []
    for curr in result[:-1]:
        curr_range = calendar.monthrange(curr.year, curr.month)
        curr_duration = (curr_range[1] - curr.day)+1
        if ((curr.month == end_date.month) & (curr.year == end_date.year)):
            durations.append(end_date.day)
        else:
            durations.append(curr_duration)
    return result[:-1],durations



In [12]:

def perform_upsell_allocation(new_data,existing_allocation,existing_monthly_allocation):
   
    df=pd.merge(left=existing_monthly_allocation,right=existing_allocation_raw,left_on=['ra_id','item'],right_on=['Ratable Allocation Id','Item ID']).sort_values('Flag', ascending=False).drop_duplicates(['item','month','year'])[['ra_id','item','month','year','revenue','flag']]
    #print(df[['item','flag']])
    print(df.shape)
    df.to_csv('D:\\vikas\\office\\castlight_v3\\generated_summary\\from_retrospective\\meta_pros_summary.csv',index=False)
    print('--------')
    new_allocation=existing_allocation[existing_allocation['Flag']==existing_allocation['Flag'].max()]
    
    current_flag=existing_allocation['Flag'].max() + 1
    print("shape is {0}".format(new_allocation.shape))
    
    #df=pd.merge(left=df,right=new_allocation,left_on='item',right_on='Item ID')[['ra_id','item','month','year','revenue','flag','Ratable Revenue','Contract Start Date']].sort_values(['item','year','month'])
    df=pd.merge(left=df,right=new_allocation,left_on='item',right_on='Item ID')[['ra_id','item','month','year','revenue','flag','Ratable Revenue','Contract Start Date']].sort_values(['item','year','month'])
    #print(df[['item','flag']])
    #df['Contract Start Date']=df['Contract Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date()) 
    df['Contract Start Date']=df['Contract Start Date'].apply(lambda x : pd.to_datetime(x, format='%m/%d/%Y').date()) 
    
    df['Contract Month']=df['Contract Start Date'].apply(lambda x: x.month)
    df['Contract Year']=df['Contract Start Date'].apply(lambda x: x.year)

    recognized_months=df[(df['month'] < upload_date.month) & (df['year'] <= upload_date.year) & (df['month'] >= df['Contract Month']) & (df['year'] >= df['Contract Year'])]
    recognized_months['flag']=current_flag
    recognized_months=recognized_months[['ra_id','item','month','year','revenue','flag']]
    print(recognized_months[['item','month']])

    recognized_revenue= df[(df['month'] < upload_date.month) & (df['year'] <= upload_date.year) & (df['month'] >= df['Contract Month']) & (df['year'] >= df['Contract Year'])].groupby('item').sum()['revenue']
    recognized_revenue=recognized_revenue.reset_index()
    #print(recognized_revenue)
    new_allocation=pd.merge(left=new_allocation,right=recognized_revenue,how='left',left_on='Item ID',right_on='item')
    new_allocation['revenue']=new_allocation['revenue'].fillna(0)
    new_allocation['Allocation Contract Value']=new_allocation['Contract Value']-new_allocation['revenue']
    
    #new_allocation['Allocation Id'] = new_allocation.apply(lambda x: str(x['Allocation Id'])[:-3]+ '-' + str('{0:0>2}'.format(x['Flag']+1)) ,axis=1 )
    new_allocation_id=new_allocation['Allocation Id'].max()
    
    new_data['Allocation Contract Value']=new_data['Contract Value']
    
    columns=['Sales Order Num','Sales Order Line ID','Orig SO Line ID','Unit Sell Price','Upload Month','Allocation Contract Value','Item ID','SSP','SSP Term','User Count','Contract Start Date','Contract End Date','Contract Value']
    
    new_allocation=new_allocation[columns]
    new_data=new_data[columns]
    new_allocation=new_allocation.append(new_data,ignore_index=True)
    
    new_allocation['Allocation Id']=new_allocation_id
#    new_allocation['Allocation Id'] = new_allocation.apply(lambda x: str(x['Allocation Id'])[:-3]+ '-' + str('{0:0>2}'.format(int(x['Flag'])+2)) ,axis=1 )
#     new_allocation['Ratable Allocation Id'] = new_allocation.apply(lambda x: str(x['Allocation Id']) + "-" + str('{0:0>2}'.format(x.name +1)), axis = 1)
    #print(new_allocation.shape)
    new_allocation['Contract Start Date']=new_allocation['Contract Start Date'].apply(lambda x : pd.to_datetime(x, format='%m/%d/%Y').date()) 
    new_allocation['Contract End Date']=new_allocation['Contract End Date'].apply(lambda x : pd.to_datetime(x, format='%m/%d/%Y').date())
    
    #new_allocation[['Upload Month']]=new_allocation[['Upload Month']].apply(lambda x : pd.to_datetime(x, format='%m/%d/%Y'))
    new_allocation['Allocation Start Date']=new_allocation['Contract Start Date'].apply(lambda x : max([x,upload_date])) 
    new_allocation['Allocation End Date']=new_allocation['Contract End Date'].apply(lambda x : x)
    
#     print(new_data[['Item ID','Contract Start Date']])
#     print(new_allocation[['Item ID','Contract Start Date','Allocation Start Date']])
    new_allocation['Allocation Term']=new_allocation.apply(lambda x: get_allocation_term(x['SSP Term'],x['Contract Start Date'],x['Contract End Date'],x['Allocation Start Date'],x['Allocation End Date']), axis=1)
    
    new_allocation['Revenue Start Date']=new_allocation['Allocation Start Date']
    new_allocation['Revenue End Date']=new_allocation['Allocation End Date']
    new_allocation['Reallocation Month']=pd.to_datetime(upload_date, format='%Y-%m-%d').date()

    new_allocation['FV Amount']=new_allocation['SSP']*new_allocation['Allocation Term']*new_allocation['User Count']

    new_allocation['EXT SSP Price']=new_allocation['FV Amount']*new_allocation['Allocation Contract Value'].sum()/new_allocation['FV Amount'].sum()
    new_allocation['Flag']=current_flag
    
    new_allocation['Ratable Revenue']=new_allocation.apply(lambda x: get_ratable_revenue(x['EXT SSP Price'], x['Revenue Start Date'],x['Revenue End Date']), axis=1)
    
    #Build allocation id and ratable allocation id
    new_allocation['Allocation Id'] = new_allocation.apply(lambda x: str(x['Allocation Id'])[:-3]+ '-' + str('{0:0>2}'.format(x['Flag']+1)) ,axis=1 )
    new_allocation['Ratable Allocation Id'] = new_allocation.apply(lambda x: str(x['Allocation Id']) + "-" + str('{0:0>2}'.format(x.name +1)), axis = 1)
    #df.to_csv('D:\\office\\castlight_v3\\generated_summary\\from_prospective\\new_alloc.csv',index=False)
    new_allocation=new_allocation[existing_allocation.columns]
    
    new_allocation['Contract Start Date']=new_allocation['Contract Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date()) 
    new_allocation['Contract End Date']=new_allocation['Contract End Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date())
    new_allocation['Allocation Start Date']=new_allocation['Allocation Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date()) 
    new_allocation['Allocation End Date']=new_allocation['Allocation End Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date())
    new_allocation['Revenue Start Date']=new_allocation['Revenue Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date()) 
    new_allocation['Revenue End Date']=new_allocation['Revenue End Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date())
    #new_allocation.to_csv('D:\\office\\castlight_v3\\generated_summary\\from_prospective\\new_pros_alloc_final.csv',index=False)
    
    new_monthly_allocation = pd.DataFrame(columns=['ra_id','item','month','year','revenue','flag']) 
    for index, row in new_allocation.iterrows():
        months, durations=get_allocation_durations(new_allocation.loc[new_allocation.index[index], 'Revenue Start Date'],new_allocation.loc[new_allocation.index[index], 'Revenue End Date'])
        for idx, val in enumerate(months):
#         for value in month_year_iter(new_allocation.loc[new_allocation.index[index], 'Revenue Start Date'].month,new_allocation.loc[new_allocation.index[index], 'Revenue Start Date'].year,new_allocation.loc[new_allocation.index[index], 'Revenue End Date'].month,new_allocation.loc[new_allocation.index[index], 'Revenue End Date'].year):  
            i=new_monthly_allocation.shape[0]
            new_monthly_allocation.loc[i,'item']=row['Item ID']
            new_monthly_allocation.loc[i,'month']=months[idx].month
            new_monthly_allocation.loc[i,'year']=months[idx].year
            new_monthly_allocation.loc[i,'revenue']=durations[idx]*row['Ratable Revenue']
            print(new_monthly_allocation.loc[i,'revenue'])
            new_monthly_allocation.loc[i,'ra_id']=row['Ratable Allocation Id']
            new_monthly_allocation.loc[i,'flag']=current_flag
    
    return new_allocation,new_monthly_allocation.append(recognized_months)

    
    
    


In [13]:
final_new_allocation = pd.DataFrame().reindex_like(existing_allocation_raw)
final_monthly_allocation = pd.DataFrame().reindex_like(existing_monthly_allocation_raw)

In [14]:
new_data_raw['Allocation ID']

11    21-21-1021-01-02
12    21-21-1021-01-02
13    21-21-1021-02-01
14    21-21-1021-03-01
Name: Allocation ID, dtype: object

In [15]:
for idx,val in enumerate(new_data_raw['Allocation ID'].unique()):
    new_data=new_data_raw[new_data_raw['Allocation ID']==val]
    existing_allocation=existing_allocation_raw[existing_allocation_raw['Allocation Id']==val].reset_index(drop=True)
    #print(existing_allocation.shape)
    final_new_allocation1, final_monthly_allocation1=perform_upsell_allocation(new_data,existing_allocation,existing_monthly_allocation=existing_monthly_allocation_raw)
    print(final_new_allocation1.shape)
    if idx == 0:
        final_monthly_allocation=final_monthly_allocation1
        final_new_allocation=final_new_allocation1
    else:
        final_monthly_allocation=final_monthly_allocation.append(final_monthly_allocation1)
        final_new_allocation=final_new_allocation.append(final_new_allocation1)
    

(104, 6)
--------
shape is (5, 25)
     item  month
35    345      4
34    345      5
0     345      6
1     345      7
81    366      4
80    366      5
73    366      6
74    366      7
103   629      4
102   629      5
94    629      6
97    629      7
14317.747163936612
13855.884352196721
14317.747163936612
13855.884352196721
14317.747163936612
14317.747163936612
13394.021540456832
14317.747163936612
4521.807875173708
4672.534804346165

c:\users\vikas\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\vikas\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4521.807875173708
4672.534804346165
4672.534804346165
4371.0809460012515
4672.534804346165
2788.630988324863
2698.675149991803
2788.630988324863
2698.675149991803
2788.630988324863
2788.630988324863
2608.7193116587428
2788.630988324863
2768.45380112676
2860.735594497652
2768.45380112676
2860.735594497652
2860.735594497652
2676.172007755868
2860.735594497652
2788.630988324863
2698.675149991803
2788.630988324863
2698.675149991803
2788.630988324863
2788.630988324863
2608.7193116587428
2788.630988324863
4994.562964163934
4833.448029836065
4994.562964163934
4994.562964163934
4672.3330955081965
4994.562964163934
7991.300742662294
7733.516847737704
7991.300742662294
7991.300742662294
7475.732952813114
7991.300742662294
(7, 25)
(104, 6)
--------
shape is (2, 25)
Empty DataFrame
Columns: [item, month]
Index: []
13561.03500761035
14013.069507864027
13561.03500761035
14013.069507864027
14013.069507864027
13561.03500761035
14013.069507864027
13561.03500761035
14013.069507864027
14013.069507864027

In [16]:
final_new_allocation['Allocation Id']

0    21-21-1021-01-03
1    21-21-1021-01-03
2    21-21-1021-01-03
3    21-21-1021-01-03
4    21-21-1021-01-03
5    21-21-1021-01-03
6    21-21-1021-01-03
0    21-21-1021-02-03
1    21-21-1021-02-03
2    21-21-1021-02-03
0    21-21-1021-03-03
1    21-21-1021-03-03
2    21-21-1021-03-03
Name: Allocation Id, dtype: object

In [17]:
final_new_allocation.to_csv('D:\\vikas\\office\\castlight_v3\\generated_summary\\from_prospective\\final_prospective_allocation_0915_v2.csv',index=False)
final_monthly_allocation.to_csv('D:\\vikas\\office\\castlight_v3\\generated_summary\\from_prospective\\final_prospective_monthly_allocation_0915_v2.csv',index=False)

In [20]:
with open('D:\\Master\\Castlight\\Data\\Standalone - TWOC\\allocation_summary1.csv', 'a', newline='') as newFile:
    newFileWriter = csv.writer(newFile)
    for index,row in final_new_allocation.iterrows()
        newFileWriter.writerow(row)

SyntaxError: invalid syntax (<ipython-input-20-34291c8adb78>, line 3)

In [205]:
with open('D:\\Master\\Castlight\\Data\\Standalone - TWOC\\monthly_allocation1.csv', 'a', newline='') as newFile:
    newFileWriter = csv.writer(newFile)
    for index,row in final_monthly_allocation.iterrows():
        newFileWriter.writerow(row)